In [ ]:
import datetime
import pandas as pd
from pprint import pprint
from sidecar import Sidecar
from ipyleaflet import Map, basemaps, DrawControl, WMSLayer, LayersControl

%pylab inline

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self,eeImageObject, visParams, name):
    mapID = ee.Image(eeImageObject).getMapId(visParams)
    tiles = WMSLayer(
    url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**mapID),
    attribution="<a href=’https://earthengine.google.com’>Google Earth Engine</a>",
    name = name,
    )
    self.add_layer(tiles)
    
Map.add_ee_layer = add_ee_layer

In [ ]:
m = Map(center=(0,35),zoom=8,basemap=basemaps.CartoDB.DarkMatter)
m.scroll_wheel_zoom = True

draw_control = DrawControl()
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "I'm sorry Dave, I'm afraid I can't do that"
    },
    "allowIntersection": False
}
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

m.add_control(draw_control)

m.add_control(LayersControl(position='topright'))

In [ ]:
sc = Sidecar(title='EE Map')

with sc:
    display(m)

In [ ]:
import ee
ee.Initialize()
import rendvi

In [ ]:
mod = rendvi.COLLECTIONS['MOD09GQ']['dataset']
mod1km = rendvi.COLLECTIONS['MOD09GA']['dataset']

In [ ]:
masked = rendvi.Masking.applyModis(mod,mod1km)
withNdvi = rendvi.Utils.addNDBand(masked,
                                   b1=rendvi.COLLECTIONS['MOD09GQ']['nir'],
                                   b2=rendvi.COLLECTIONS['MOD09GQ']['red'],
                                   outName='ndvi')

In [ ]:
comp = ee.Image(withNdvi.first())

m.add_ee_layer(comp,{'bands':"ndvi",'min':-0.2,'max':1,'palette':'gray,beige,green,darkgreen'},'NDVI')

In [ ]:
# time information to handle image collection
iniYear = 2000
endYear = 2019

today = ee.Date(datetime.datetime.now().strftime('%Y-%m-%d')).advance(-7,'day')

# convert start and end dates to EE date objects
eeIni = ee.Date.fromYMD(iniYear,1,1,)
eeEnd = ee.Date.fromYMD(endYear,12,31)

# make list of years for loop processing
years = ee.List.sequence(iniYear,endYear)

In [ ]:
full = rendvi.Processing(withNdvi,'ndvi')

In [ ]:
dekads = full.getDekadImages(years)

In [ ]:
despiked = dekads.applyDespike(window=30,step=10)

x = ee.Image(despiked.IC.first()).reproject(ee.Projection('epsg:4326').atScale(250))
m.add_ee_layer(x,{'bands':"ndvi",'min':-0.2,'max':1,'palette':'gray,beige,green,darkgreen'},'Despiked')

In [ ]:
climo = despiked.calcClimatology()

climo = climo.map(lambda x: x.divide(10000).set('system:time_start',x.get('system:time_start')))

In [ ]:
backFilled = despiked.climatologyBackFill(climo)

x = ee.Image(backFilled.imageCollection.first()).reproject(ee.Projection('epsg:4326').atScale(250))
m.add_ee_layer(x,{'bands':"ndvi",'min':-0.2,'max':1,'palette':'gray,beige,green,darkgreen'},'Back filled')

In [ ]:
smoothed = backFilled.applySmoothing(window=30)

In [ ]:
final = ee.Image(smoothed.imageCollection.first())

m.add_ee_layer(final,{'bands':"ndvi",'min':-0.2,'max':1,'palette':'Gray,beige,green,darkgreen'},'Smoothed')

In [ ]:
# geo = ee.Geometry.Point([35,0])

# x = final.filterDate("2017-01-01","2018-02-01").getRegion(geo,250).getInfo()

In [ ]:
# df = pd.DataFrame(x[1:])
# df.columns = x[0]


In [ ]:
# df.head()

In [ ]:
# x.getInfo()

In [ ]:
exportRegion = ee.Geometry.Rectangle([-180,-60,180,85],'epsg:4326',False)

In [ ]:
!earthengine --no-use_cloud_api task list